In [ ]:
import os
from joblib import Parallel,delayed
import pandas as pd
from tqdm import tqdm
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss,classification_report,f1_score
import time
import gc
import math
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import joblib
import numpy as np
import pickle,os
from scipy.stats import entropy
input_dir='../data/BAAI-INSPEC-Industrial-Quality-Prediction-Challenge_107/'
a=os.listdir(input_dir+'INSPEC_train')
def read_csv(idx):
    wb = xlrd.open_workbook(input_dir+'INSPEC_train/'+a[idx])
    sheets = wb.sheet_names()
    df1 = pd.read_excel(input_dir+'INSPEC_train/'+a[idx], sheet_name=sheets[0])
    df2 = pd.read_excel(input_dir+'INSPEC_train/'+a[idx], sheet_name=sheets[1])
    df3 = pd.read_excel(input_dir+'INSPEC_train/'+a[idx], sheet_name=sheets[2])
    df = df2.merge(df1,on=['ID_F_PROCESS'],how='left')
    df = df3.merge(df,on =['ID_F_PHASE','ID_F_PHASE_S'],how = 'left')
    return df
dfs = Parallel(n_jobs=-1, verbose=10)(delayed(read_csv)(idx) for idx in range(len(a)))
train = pd.concat(dfs, axis=0)
train.reset_index(drop=True, inplace=True)
train.to_csv(input_dir+'train.csv',index=False)
## Function to reduce the memory usage
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

train = pd.read_csv('../kernel/train.csv')
train.rename(columns={'ID_F_PROCESS':'Product_ID'}, inplace=True)
bad_id = train[(train['ID_F_PHASE_S'] < 3)&(train['PHASE_RESULT_STATE'] == 2)]['Product_ID'].unique()
train1 = train[(train['ID_F_PHASE_S'] < 3)&(~train['Product_ID'].isin(bad_id))]
train1.dropna(subset=['PROCESS_RESULT_STATE'],inplace =True)
#train1 = train1[train1['PRODUCTGROUP_NAME'].isin(test1['PRODUCTGROUP_NAME'].unique())]
test1 = pd.read_csv(input_dir+'final_predict_4_no_label.csv')
df =train1.append(test1).reset_index(drop=True)

#构造特征
cat_cols, num_cols = [], []

for col in df.columns:
    if df[col].dtype == object:
        cat_cols.append(col)
    else:
        num_cols.append(col)
print('Numerical columns {}, categorical columns {}'.format(len(num_cols), len(cat_cols)))
#添加side和axis的键
df['side_axis'] = df['SIDE'].astype('str')+df['AXIS'].astype('str')
df['side_axis'] = df['side_axis'].map(dict(zip(df['side_axis'].unique(),range(df['side_axis'].nunique()))))
cat_cols.remove('Product_ID')
cat_cols.remove('RESULT_STRING')
cat_cols.remove('SIDE')
cat_cols.append('ID_F_PHASE_S')
cat_cols.append('ID_F_PARAMETER_S')
cat_cols.append('side_axis')
#编码
encode_col = ['AXIS','PHASE_NAME','PRODUCTGROUP_NAME','RESULT_STRING','SIDE','TYPE_NUMBER','ID_F_PHASE']
for i in tqdm(encode_col):
    df[i] = df[i].map(dict(zip(df[i].unique(),range(df[i].nunique()))))
df = reduce_mem_usage(df)
df['cha'] = df['RESULT_VALUE'] - df['LOWER_LIMIT'] / (df['UPPER_LIMIT'] - df['LOWER_LIMIT'])
stat_cols = ['RESULT_VALUE','UPPER_LIMIT','LOWER_LIMIT']
#df['cha'] = df['UPPER_LIMIT']-df['LOWER_LIMIT']

#COUNT特征
for f in tqdm(cat_cols):
    print(f)
    map_dict = dict(zip(df[f].unique(), range(df[f].nunique())))
    df[f] = df[f].map(map_dict).fillna(-1).astype('int32')
    df[f + '_count'] = df[f].map(df[f].value_counts())

#countrank特征
def get_most(df,col):
    for i in col:
        if i =='ID_F_PHASE':
            continue
        fit_dict = {}
        counter =  Counter(df[i]).most_common()
        fit_dict[i] = {k: j for (j, (k, v)) in enumerate(counter)}
        df[i+'_num_countrank'] = df[i].map(fit_dict[i])
    return df
df = get_most(df,cat_cols)
for f in tqdm(cat_cols):
    tmp = df.groupby([f])
    for col in (cat_cols):
        if col == f or  f == 'ID_F_PHASE' or col =='ID_F_PHASE' or f == 'Product_ID' or col =='Product_ID':
            continue
            print(col)
        df['{}_{}_nunique'.format(f, col)]= tmp[col].transform('nunique')
        df['{}_{}_mode'.format(f, col)]= tmp[col].transform(lambda x: x.mode()[0])
        df['{}_{}_count'.format(f, col)]= df.groupby([f, col])['Product_ID'].transform('count')
        df['{}_{}_count_ratio'.format(col, f)] = df['{}_{}_count'.format(f, col)] / df[f + '_count'] # 比例偏好
        df['{}_{}_count_ratio'.format(f, col)] = df['{}_{}_count'.format(f, col)] / df[col + '_count'] # 比例偏好
        df['{}_{}_nunique_ratio_{}_count'.format(f, col, f)] = df['{}_{}_nunique'.format(f, col)] / df[f + '_count']
        df['{}_{}_mode_count'.format(f, col)] = df['{}_{}_mode'.format(f, col)].map(df['{}_{}_mode'.format(f, col)].value_counts())
        # mode_cols.append('{}_{}_mode'.format(f, col))
    del tmp
    gc.collect()
#统计特征
def trans(df,group_col,stat_col):
    for i in tqdm(group_col):
        for j in stat_col:
            df[i+'_'+j+'_mean'] = df.groupby([i])[j].transform('mean')
            df[i+'_'+j+'_max'] = df.groupby([i])[j].transform('max')
            df[i+'_'+j+'_min'] = df.groupby([i])[j].transform('min')
            df[i+'_'+j+'_medain'] = df.groupby([i])[j].transform('median')
            df[i+'_'+j+'_std'] = df.groupby([i])[j].transform('std')
    return df
df = trans(df,cat_cols,stat_cols)
#数据划分
feature_names = [i for i in df.columns if i not in['PROCESS_RESULT_STATE','Product_ID','ID_F_PROCESS','ID_F_PHASE_num_countrank'] and 'Product_ID' not in i]
dic={1:1,2:0}
df['PROCESS_RESULT_STATE'] = df['PROCESS_RESULT_STATE'].map(dic)
trn_index = ~df.PROCESS_RESULT_STATE.isnull()
train_1 = df[trn_index][feature_names]
y1   = df[trn_index]['PROCESS_RESULT_STATE']
test1 = df[~trn_index][feature_names]
print(train_1.shape,test1.shape)
#val1
#五折交叉验证
param_new={
    'objective':'binary',
 'bagging_fraction': 0.9,
 'feature_fraction': 0.8,
 'learning_rate': 0.05,
 'num_leaves': 127,
#  'is_unbalance':'True',
    'metric': {'auc'},
}
lgb_oof = np.zeros((len(train_1),))
lgb_prediction = np.zeros((len(test1),))
seeds = [516,3056,2028,2369,2050]
num_model_seed = 1
print('training')
for model_seed in range(num_model_seed):
    oof_lgb = np.zeros((len(train_1),))
    prediction_lgb=np.zeros((len(test1),))
    skf = StratifiedKFold(n_splits=5, random_state=seeds[model_seed], shuffle=True)
 #   skf= GroupKFold(n_splits = 5)
    for fold_, (trn_idx, val_idx) in enumerate(skf.split(train_1, y1)):
        print("fold n°{}".format(fold_+1))
        trn_data = lgb.Dataset(train_1.iloc[trn_idx], y1.iloc[trn_idx])
        val_data = lgb.Dataset(train_1.iloc[val_idx], y1.iloc[val_idx])
        num_round = 5000
        clf1 = lgb.train(param_new, 
                        trn_data, 
                        num_round,
                        #categorical_feature = cat_cols,
                        valid_sets = [trn_data, val_data], 
                        verbose_eval = 200,
                       early_stopping_rounds=200,
                       )
        oof_lgb[val_idx] = clf1.predict(train_1.iloc[val_idx], num_iteration=clf1.best_iteration)

        prediction_lgb += clf1.predict(test1, num_iteration=clf1.best_iteration) / skf.n_splits
        gc.collect()
    lgb_oof += oof_lgb/ num_model_seed
    lgb_prediction += prediction_lgb / num_model_seed
    print('auc',roc_auc_score(y1, oof_lgb))
print('auc',roc_auc_score(y1, lgb_oof))
test1 = pd.read_csv(input_dir+'final_predict_4_no_label.csv')
sub1 = pd.DataFrame()
sub1['id'] = test1['Product_ID']
sub1['label_'] = lgb_prediction
sub1['label'] = sub1.groupby(['id'])['label_'].transform('mean')
sub1_last = sub1[['id','label']].drop_duplicates('id').reset_index(drop=True)
sub1_last['label'] = sub1_last['label'].apply(lambda x: 1 if x >= 0.965 else 0)
print('前三步：',sub1_last['label'].value_counts())

print("-----------------------qian shi bu--------------------------------------")
bad_id = train[(train['ID_F_PHASE_S'] < 10)&(train['PHASE_RESULT_STATE'] == 2)]['Product_ID'].unique()
train1 = train[(train['ID_F_PHASE_S'] < 10)&(~train['Product_ID'].isin(bad_id))]
train1.dropna(subset=['PROCESS_RESULT_STATE'],inplace =True)
#train1 = train1[train1['PRODUCTGROUP_NAME'].isin(test2['PRODUCTGROUP_NAME'].unique())]
test2 = pd.read_csv(input_dir+'final_predict_11_no_label.csv')
df =train1.append(test2).reset_index(drop=True)
#构造特征
cat_cols, num_cols = [], []

for col in df.columns:
    if df[col].dtype == object:
        cat_cols.append(col)
    else:
        num_cols.append(col)
print('Numerical columns {}, categorical columns {}'.format(len(num_cols), len(cat_cols)))
#添加side和axis的键
df['side_axis'] = df['SIDE'].astype('str')+df['AXIS'].astype('str')
df['side_axis'] = df['side_axis'].map(dict(zip(df['side_axis'].unique(),range(df['side_axis'].nunique()))))
#cat_cols.remove('Product_ID')
cat_cols.remove('RESULT_STRING')
cat_cols.remove('SIDE')
cat_cols.append('ID_F_PHASE_S')
cat_cols.append('ID_F_PARAMETER_S')
cat_cols.append('side_axis')
#编码
encode_col = ['AXIS','PHASE_NAME','PRODUCTGROUP_NAME','RESULT_STRING','SIDE','TYPE_NUMBER','ID_F_PHASE']
for i in tqdm(encode_col):
    df[i] = df[i].map(dict(zip(df[i].unique(),range(df[i].nunique()))))
df = reduce_mem_usage(df)
stat_cols = ['RESULT_VALUE','UPPER_LIMIT','LOWER_LIMIT']
df['cha'] = df['UPPER_LIMIT']-df['LOWER_LIMIT']
#COUNT特征
for f in tqdm(cat_cols):
    print(f)
    map_dict = dict(zip(df[f].unique(), range(df[f].nunique())))
    df[f] = df[f].map(map_dict).fillna(-1).astype('int32')
    df[f + '_count'] = df[f].map(df[f].value_counts())

#countrank特征
def get_most(df,col):
    for i in col:
        if i =='ID_F_PHASE':
            continue
        fit_dict = {}
        counter =  Counter(df[i]).most_common()
        fit_dict[i] = {k: j for (j, (k, v)) in enumerate(counter)}
        df[i+'_num_countrank'] = df[i].map(fit_dict[i])
    return df
df = get_most(df,cat_cols)
# #类别特征
for f in tqdm(cat_cols):
    tmp = df.groupby([f])
    for col in (cat_cols):
        if col == f or  f == 'ID_F_PHASE' or col =='ID_F_PHASE' or f == 'Product_ID' or col =='Product_ID':
            continue
            print(col)
        df['{}_{}_nunique'.format(f, col)]= tmp[col].transform('nunique')
        df['{}_{}_mode'.format(f, col)]= tmp[col].transform(lambda x: x.mode()[0])
        df['{}_{}_count'.format(f, col)]= df.groupby([f, col])['Product_ID'].transform('count')
        df['{}_{}_count_ratio'.format(col, f)] = df['{}_{}_count'.format(f, col)] / df[f + '_count'] # 比例偏好
        df['{}_{}_count_ratio'.format(f, col)] = df['{}_{}_count'.format(f, col)] / df[col + '_count'] # 比例偏好
        df['{}_{}_nunique_ratio_{}_count'.format(f, col, f)] = df['{}_{}_nunique'.format(f, col)] / df[f + '_count']
        df['{}_{}_mode_count'.format(f, col)] = df['{}_{}_mode'.format(f, col)].map(df['{}_{}_mode'.format(f, col)].value_counts())
        # mode_cols.append('{}_{}_mode'.format(f, col))
    del tmp
    gc.collect()
#统计特征
def trans(df,group_col,stat_col):
    for i in tqdm(group_col):
        for j in stat_col:
            df[i+'_'+j+'_mean'] = df.groupby([i])[j].transform('mean')
            df[i+'_'+j+'_max'] = df.groupby([i])[j].transform('max')
            df[i+'_'+j+'_min'] = df.groupby([i])[j].transform('min')
            df[i+'_'+j+'_medain'] = df.groupby([i])[j].transform('median')
            df[i+'_'+j+'_std'] = df.groupby([i])[j].transform('std')
    return df
df = trans(df,cat_cols,stat_cols)
#数据划分
feature_names = [i for i in df.columns if i not in['PROCESS_RESULT_STATE','Product_ID','ID_F_PROCESS','ID_F_PHASE_num_countrank']and 'Product_ID' not in i]
dic={1:1,2:0}
df['PROCESS_RESULT_STATE'] = df['PROCESS_RESULT_STATE'].map(dic)
trn_index = ~df.PROCESS_RESULT_STATE.isnull()
train_1 = df[trn_index][feature_names]
y1   = df[trn_index]['PROCESS_RESULT_STATE']
test2 = df[~trn_index][feature_names]
print(train_1.shape,test2.shape)
#val2
#五折交叉验证
param_new={
    'objective':'binary',
 'bagging_fraction': 0.7808187852160583,
 'feature_fraction': 0.3043366780447281,
 'lambda_l1': 0.25,
 'lambda_l2': 0.4,
 'learning_rate': 0.06259858514656305,
 'max_bins': 59,
 'min_data_in_leaf': 21,
 'num_leaves': 102,
#  'is_unbalance':'True',
    'metric': {'auc'},
}
lgb_oof = np.zeros((len(train_1),))
lgb_prediction1 = np.zeros((len(test2),))
seeds = [516,3056,2028,2369,2050]
num_model_seed = 1
print('training')
for model_seed in range(num_model_seed):
    oof_lgb = np.zeros((len(train_1),))
    prediction_lgb1=np.zeros((len(test2),))
    skf = StratifiedKFold(n_splits=5, random_state=seeds[model_seed], shuffle=True)
 #   skf= GroupKFold(n_splits = 5)
    for fold_, (trn_idx, val_idx) in enumerate(skf.split(train_1, y1)):
        print("fold n°{}".format(fold_+1))
        trn_data = lgb.Dataset(train_1.iloc[trn_idx], y1.iloc[trn_idx])
        val_data = lgb.Dataset(train_1.iloc[val_idx], y1.iloc[val_idx])
        num_round = 5000
        clf1 = lgb.train(param_new, 
                        trn_data, 
                        num_round,
                        #categorical_feature = cat_cols,
                        valid_sets = [trn_data, val_data], 
                        verbose_eval = 200,
                       early_stopping_rounds=200,
                       )
        oof_lgb[val_idx] = clf1.predict(train_1.iloc[val_idx], num_iteration=clf1.best_iteration)

        prediction_lgb1 += clf1.predict(test2, num_iteration=clf1.best_iteration) / skf.n_splits
        gc.collect()
    lgb_oof += oof_lgb/ num_model_seed
    lgb_prediction1 += prediction_lgb1 / num_model_seed
    print('auc',roc_auc_score(y1, oof_lgb))
print('auc',roc_auc_score(y1, lgb_oof))
test2 = pd.read_csv(input_dir+'final_predict_11_no_label.csv')
sub2 = pd.DataFrame()
sub2['id'] = test2['Product_ID']

sub2['label_'] = lgb_prediction1
sub2['label'] = sub2.groupby(['id'])['label_'].transform('mean')
sub2_last = sub2[['id','label']].drop_duplicates('id').reset_index(drop=True)
sub2_last['label'] = sub2_last['label'].apply(lambda x: 1 if x >=0.97 else 0)
print('前十步：',sub2_last['label'].value_counts())
sub = sub1_last.append(sub2_last).reset_index(drop=True)
d = {0:2,1:1}
sub['label'] = sub['label'].map(d)
sub.to_csv('../work/submission.csv',index=False)
print('result',sub.label.value_counts())
